Currently running to optimize the geometry of 12 detectors

In [8]:
three=False
four=False
five=False
six=False
seven=False
eight=False
twelve=True

In [9]:

import time
import numpy as np
import matplotlib.pyplot as plt
import math as mth
import random as rand
import statistics as s
import healpy as hp
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.basemap import Basemap
from matplotlib import cm, colors


In [10]:
#Some functions I used determine whether or not a signal could even be found in the detector

def length(v):
    return mth.sqrt(np.dot(v, v))

def angle(v1, v2):

    return mth.acos(np.dot(v1, v2) / (length(v1) * length(v2)))

def frodo(theta):
    #Creates rings around every 10 degree separation for source positions, used to make a weighted average for optimizing geos. 
    if theta==0: 
        #have to treat this slightly differently
        A=abs(2*np.pi*(np.cos(theta)-np.cos(np.deg2rad(5)))) 
    else:
        A=abs(2*np.pi*(np.cos(theta+np.deg2rad(5))-np.cos(theta-np.deg2rad(5))))
    return A

def chimaker(chiterms,Ndets): 
        #after looping throug every single chi term, combine them into an array of the chi squared of every possible source. 
    if three==True:
        a,b,c=np.array_split(chiterms,Ndets)
        ab=np.add(a,b)
        chisquareds=(np.add(ab,c)) 
    if four==True:
        a,b,c,d=np.array_split(chiterms,Ndets)
        ab=np.add(a,b)
        cd=np.add(c,d)
        chisquareds=(np.add(ab,cd)) 
    if five==True:
        a,b,c,d,e=np.array_split(chiterms,Ndets)
        ab=np.add(a,b)
        cd=np.add(c,d)
        ad=np.add(ab,cd)
        chisquareds=(np.add(ad,e)) 
    if six==True:
        a,b,c,d,e,f=np.array_split(chiterms,Ndets)
        ab=np.add(a,b)
        cd=np.add(c,d)
        ad=np.add(ab,cd)
        ae=np.add(ad,e)
        chisquareds=(np.add(ae,f))
    if eight==True:
        a,b,c,d,e,f,g,h=np.array_split(chiterms,Ndets)
        ab=np.add(a,b)
        cd=np.add(c,d)
        ef=np.add(e,f)
        gh=np.add(g,h)
        ad=np.add(ab,cd)
        eh=np.add(ef,gh)
        ah=np.add(ad,eh)
        chisquareds=ah
    if twelve==True:
        a,b,c,d,e,f,g,h,i,j,k,l=np.array_split(chiterms,Ndets)
        ab=np.add(a,b)
        cd=np.add(c,d)
        ef=np.add(e,f)
        gh=np.add(g,h)
        ij=np.add(i,j)
        kl=np.add(k,l)
        
        ad=np.add(ab,cd)
        eh=np.add(ef,gh)
        il=np.add(ij,kl)
        ah=np.add(ad,eh)
        al=np.add(ah,il)
        chisquareds=al
    return chisquareds

        
def solver(detsvals,bottheta,toptheta,botphi,topphi,n):
#this function uses a chi squared minimizer over a given range to identify the theta,phi,
#and Ao values which correspond to the minimum chi squared and thus localized source. 
    confidence = []
    chiterms = []
    thecon = []
    phicon = []
    for s in range(len(detsvals)):     
        oa=np.deg2rad(np.linspace(bottheta,toptheta,n/4))  #range of thetas to sample
        ob=np.deg2rad(np.linspace(botphi,topphi,n)) #phi
        Aofit=np.linspace(0,1000,25)  
        for sa in range(len(oa)): 
            for sb in range(len(ob)):
                for sc in range(len(Aofit)):
                    #make sure it fits within the acceptable range
                    if oa[sa]>=0 and oa[sa]<=np.pi and ob[sb]>=0 and ob[sb]<=2*np.pi:
                        CHIsourceang=[oa[sa],ob[sb]]
                        CHIsourcexyz = hp.ang2vec(CHIsourceang[0],CHIsourceang[1])
                        CHIsep=angle(CHIsourcexyz,dets[s])                                           
                        if CHIsep<np.pi: 
                            chi=Aofit[sc]*np.dot(CHIsourcexyz,dets[s])+bg
                        else:
                            chi=0            
                        if detvals[s]>0:   #if there is a signal in the detector 
                            chiterm=((chi-detsvals[s])**2/detsvals[s])
                        else:    #if not, just zero 
                            chiterm=10000
                    else: 
                        chiterm=10000 #some large #, just to note that it definitely isn't right angle 
                      #  print("nothing to see here, move along")
                                                                       
                                    
                    chiterms.append(chiterm)   #this is an array of EVERY SINGLE term, need to split in pieces and add element by element...
    chisquareds=chimaker(chiterms,len(detsvals))
    
    chimin=np.amin(chisquareds)
    chisquareds=list(chisquareds)
    thetaloc = np.rad2deg(oa[int((chisquareds.index(chimin)-(chisquareds.index(chimin) % (len(ob)*len(Aofit))))/(len(ob)*len(Aofit)))])
    philoc = np.rad2deg(ob[int(((chisquareds.index(chimin) % (len(ob)*len(Aofit)))-(chisquareds.index(chimin) % (len(Aofit))))/len(Aofit))])
    Aoguess=Aofit[int((chisquareds.index(chimin) % (len(ob)*len(Aofit)))  % len(Aofit))]
    #print(Aoguess)
    #print(thetaloc,philoc)
    
    return thetaloc,philoc,chisquareds,chimin,Aoguess

In [11]:
start_time = time.time()

zenith=[0,0]  #set zenith of the system

#Test detector orientations(letters B,C,E,etc. are at same but oppiste side angles)

#These are all the optimal tilts for the detectors already solved, could easily add more if wanted.. 
if three==True:
    tiltA=np.deg2rad(np.array([15]))    #3 detectors
if four==True:    
    tiltA=np.deg2rad(np.array([35]))    #4 detectors
if five==True:
    tiltA=np.deg2rad(np.array([45]))    #5 detectors
if six==True:
    tiltA=np.deg2rad(np.array([45]))    #6 detectors
if eight==True:
    tiltA=np.deg2rad(np.array([45]))    #8 detectors
if twelve==True:
    tiltA=np.deg2rad(np.array([45]))

tiltD1=np.deg2rad(np.array([30]))    #linspace(30,75,4)

#Test source locations
thetaS=np.deg2rad(np.linspace(0,60,7)) #so starting at zenith to 60 degrees tilted, testing points every 10 degrees

phiS=np.deg2rad(np.linspace(0,360,16))    #along each of these zenithal angles, sampling 16 points along their respective azimuths. 


In [12]:
Ao=np.array([500]) #intensity of the signal 
bg=1000
error=[]
x = 100   #run through x times per setup
delta=5  #some window to test a more precise loop of possible signals
thetaRMS = [] 
phiRMS = []
chimins = []   #create a list of every minimized chisqu
thetalocs = []
philocs = []
eachgeo = []  #average localization offset of each geometry 

In [13]:
for ja in range(len(tiltA)):   #run through the possible detector orientations
    #should include another loop starting here for the other tilt direciton (D)
    tiltB=tiltA[ja]
    if three==True:
                  
        tiltC=tiltA[ja]
    if four==True:
        tiltB=tiltA[ja]

    if five==True:
      
        tiltC=tiltB
        tiltD=tiltB
        tiltE=tiltB
    if six==True:
       
        tiltC=tiltB
        tiltD=tiltB
        tiltE=tiltB
        tiltF=tiltB
    
    if eight==True:
        tiltG=tiltE=tiltC=tiltA[ja]
####################################################
    if twelve==True: 
        tiltK=tiltI=tiltG=tiltE=tiltC=tiltA[ja]
#####################################################
    for jb in range(len(tiltD1)): 
        print("Detector Tilt " + str(np.rad2deg(tiltA[ja]))+ " degrees x " +str(np.rad2deg(tiltD1[jb]))+ " degrees")
        if four==True:
            tiltE=tiltD1[jb]
        if twelve==True: #for alternating tilt designs.. run thru this tomorrow!!!!!!
            tiltL=tiltJ=tiltH=tiltF=tiltD=tiltD1[jb]
        if eight==True:
            tiltH=tiltF=tiltD=tiltD1[jb]
        for k in range(len(Ao)):  #(optional) run through different signal strengths
            angoffset = []   #rms vals for every theta, phi 
            totarea=0 
            azavg = []  #average RMS within each azimuth position, aka within a ring 
            ringareas = []   #all of the areas of the rings
            for ia in range(len(thetaS)):  #run through the possible source angles
                #area of ring here 
                ringsize = frodo(thetaS[ia])  
                ringareas.append(ringsize)
                totarea += ringsize  #don't really need this, just to confirm it's doing the right thing!
                ringmeansep = 0  #resets for each
                for ib in range(len(phiS)):
                    thetalocs = []
                    philocs = []
                    Aoguesses = []
                    angseps = []
                    sourceang=[thetaS[ia],phiS[ib]]
                    sourcexyz = hp.ang2vec(sourceang[0],sourceang[1])

                    if three==True: 
                        Anorm = hp.ang2vec(zenith[0]+tiltA[ja],zenith[1])
                        Bnorm = hp.ang2vec(zenith[0]+tiltB,zenith[1]+2*np.pi/3)
                        Cnorm = hp.ang2vec(zenith[0]+tiltC,zenith[1]+4*np.pi/3)

                        dets = [Anorm,Bnorm,Cnorm]
                    if four==True:
                        Anorm = hp.ang2vec(zenith[0]+tiltA[ja],zenith[1])
                        Bnorm = hp.ang2vec(zenith[0]+tiltB,zenith[1]+np.pi)
                        Cnorm = hp.ang2vec(zenith[0]+tiltC,zenith[1]+np.pi/2)
                        Dnorm = hp.ang2vec(zenith[0]+tiltD[jb],zenith[1]+3*np.pi/2)
                        dets = [Anorm,Bnorm,Cnorm,Dnorm] 
                    if five==True:
                        Anorm = hp.ang2vec(zenith[0]+tiltA[ja],zenith[1]+1.25664)
                        Bnorm = hp.ang2vec(zenith[0]+tiltB,zenith[1]+2*1.25664)
                        Cnorm = hp.ang2vec(zenith[0]+tiltC,zenith[1]+3*1.25664)
                        Dnorm = hp.ang2vec(zenith[0]+tiltD,zenith[1]+4*1.25664)
                        Enorm = hp.ang2vec(zenith[0]+tiltE,zenith[1]+5*1.25664)
                   
                    
                        dets = [Anorm,Bnorm,Cnorm,Dnorm,Enorm]   
                    if six==True:
                        Anorm = hp.ang2vec(zenith[0]+tiltA[ja],zenith[1]+np.pi/3)
                        Bnorm = hp.ang2vec(zenith[0]+tiltB,zenith[1]+2*np.pi/3)
                        Cnorm = hp.ang2vec(zenith[0]+tiltC,zenith[1]+3*np.pi/3)
                        Dnorm = hp.ang2vec(zenith[0]+tiltD,zenith[1]+4*np.pi/3)
                        Enorm = hp.ang2vec(zenith[0]+tiltE,zenith[1]+5*np.pi/3)
                        Fnorm = hp.ang2vec(zenith[0]+tiltE,zenith[1])  
                    
                        dets = [Anorm,Bnorm,Cnorm,Dnorm,Enorm,Fnorm] 
                    if eight==True:
                        
                        Anorm = hp.ang2vec(zenith[0]+tiltA[ja],zenith[1])
                        Bnorm = hp.ang2vec(zenith[0]+tiltD1[jb],zenith[1]+np.pi/4)
                        Cnorm = hp.ang2vec(zenith[0]+tiltC,zenith[1]+np.pi/2)
                        Dnorm = hp.ang2vec(zenith[0]+tiltD,zenith[1]+3*np.pi/4)
                        Enorm = hp.ang2vec(zenith[0]+tiltE,zenith[1]+np.pi)
                        Fnorm = hp.ang2vec(zenith[0]+tiltF,zenith[1]+5*np.pi/4)
                        Gnorm = hp.ang2vec(zenith[0]+tiltG,zenith[1]+3*np.pi/2)
                        Hnorm = hp.ang2vec(zenith[0]+tiltH,zenith[1]+7*np.pi/4)
                        dets = [Anorm,Bnorm,Cnorm,Dnorm,Enorm,Fnorm,Gnorm,Hnorm] 
                    
                    if twelve==True: 
                        Anorm = hp.ang2vec(zenith[0]+tiltA[ja],zenith[1])
                        Bnorm = hp.ang2vec(zenith[0]+tiltD1[jb],zenith[1]+np.pi/6)
                        Cnorm = hp.ang2vec(zenith[0]+tiltC,zenith[1]+2*np.pi/6)
                        Dnorm = hp.ang2vec(zenith[0]+tiltD,zenith[1]+3*np.pi/6)
                        Enorm = hp.ang2vec(zenith[0]+tiltE,zenith[1]+4*np.pi/6)
                        Fnorm = hp.ang2vec(zenith[0]+tiltF,zenith[1]+5*np.pi/6)
                        Gnorm = hp.ang2vec(zenith[0]+tiltG,zenith[1]+6*np.pi/6)
                        Hnorm = hp.ang2vec(zenith[0]+tiltH,zenith[1]+7*np.pi/6)
                        Inorm = hp.ang2vec(zenith[0]+tiltI,zenith[1]+8*np.pi/6)
                        Jnorm = hp.ang2vec(zenith[0]+tiltJ,zenith[1]+9*np.pi/6)
                        Knorm = hp.ang2vec(zenith[0]+tiltK,zenith[1]+10*np.pi/6)
                        Lnorm = hp.ang2vec(zenith[0]+tiltL,zenith[1]+11*np.pi/6)
                        dets = [Anorm,Bnorm,Cnorm,Dnorm,Enorm,Fnorm,Gnorm,Hnorm,Inorm,Jnorm,Knorm,Lnorm]   

                    count=0
                    while count<x: 
        
                        detvals = []
                        for ii in range(len(dets)):
                            sep=angle(sourcexyz,dets[ii])
                            if sep<np.pi/2:
                                dexp=Ao[k]*np.dot(sourcexyz,dets[ii])  #expected number of counts in A
                            else:
                                dexp=0
                                #print("Warning: expected signal not found in detector "+str(ii))
                            counts=max(dexp,0)+bg
                            unccounts=np.sqrt(counts)
                            det=rand.gauss(counts,unccounts)
                            if det-bg<0:
                                det=0
                                #print("Warning: no signal found in detector "+str(ii))
                            #talking too much, had to mute
                            detvals.append(det)  #each detector with noise, need this array for solver
                        coarsethetaloc,coarsephiloc,coarsechisquareds,cmin,coarseAo = solver(detvals,0,90,0,360,20)  #runs through big range with few points, easy to pinpoint from here
                        finethetaloc,finephiloc,finechisquareds,fmin,fineAo = solver(detvals,coarsethetaloc-delta,coarsethetaloc+delta,coarsephiloc-delta,coarsephiloc+delta,12)                      
                        chimins.append(fmin)
                        Aoguesses.append(fineAo)
                        acvec=hp.ang2vec(np.deg2rad(finethetaloc),np.deg2rad(finephiloc)) #actual vector position (on unit circle)  of the source for trial x-n
                        angsep = angle(sourcexyz,acvec)
                        angseps.append(angsep)   #array of how far apart they
                        #print(np.rad2deg(angsep))
                       
                        count += 1     
                    meansep=s.mean(np.rad2deg(angseps))
                    ringmeansep+=meansep  #reset this before every theta
                    angoffset.append(meansep)
                azavg.append(ringmeansep/len(phiS))
                #is ringSTD azimuthal average? let's double check. 
                #print((ringSTD))
        #so here I have all the stuff done, now I need to weight each point 
                print("starting the next theta...")  #useful as a way to keep track of time... 
        
        weighted = []    
        for il in range(len(azavg)):         
        #multiply each number by its weight, divide by its total sum, add up
            weighted.append(azavg[il]*ringareas[il])
        print(np.sum(weighted)/np.sum(ringareas))
        eachgeo.append(np.sum(weighted)/np.sum(ringareas)) 
           # for every test source in every ring for every theta, average these out to find a weighted avg of how valuable they are
            
            
            #prints out the standard deviation at each tilt, and has it weighted based on how large of an area this represents       
end_time = time.time()
print(eachgeo)
print("Time elapsed: "+ str(end_time-start_time))                    

Detector Tilt 30.0 degrees x 30.0 degrees
hey1
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
4.71435517464
Detector Tilt 30.0 degrees x 45.0 degrees
hey1
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
4.96697730986
Detector Tilt 45.0 degrees x 30.0 degrees
hey1
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
5.21930195368
Detector Tilt 45.0 degrees x 45.0 degrees
hey1
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
starting the next theta...
5.60139115246


The if statements below could be used instead of a weighted average. Throw these inside the code just after the thetaS for loop, and should hypothetically sample points evenly all the way down to 90 degrees below zenith. 

In [ ]:

if thetaS[ia]==np.deg2rad(0):
    phiS=np.deg2rad(np.linspace(0,360,1))
if thetaS[ia]==np.deg2rad(10):
    phiS=np.deg2rad(np.linspace(0,360,7))
if thetaS[ia]==np.deg2rad(20):
    phiS=np.deg2rad(np.linspace(0,360,14))
if thetaS[ia]==np.deg2rad(30):
    phiS=np.deg2rad(np.linspace(0,360,21))
if thetaS[ia]==np.deg2rad(40):
    phiS=np.deg2rad(np.linspace(0,360,27))
if thetaS[ia]==np.deg2rad(50):
    phiS=np.deg2rad(np.linspace(0,360,32))
if thetaS[ia]==np.deg2rad(60):
    phiS=np.deg2rad(np.linspace(0,360,37))
if thetaS[ia]==np.deg2rad(70):
    phiS=np.deg2rad(np.linspace(0,360,40))
if thetaS[ia]==np.deg2rad(80):
    phiS=np.deg2rad(np.linspace(0,360,42))
if thetaS[ia]==np.deg2rad(90):
    phiS=np.deg2rad(np.linspace(0,360,42))